In [130]:
# Bibliotecas de Manipulação de dados 
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
# Bibliotecas de machine Learning
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
# Bibliotecas de métricas de avaliação
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
df_titanic_train = pd.read_csv("C:/Users/User/Downloads/train.csv")
titanic_test = pd.read_csv("C:/Users/User/Downloads/test.csv")

In [3]:
df_titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [219]:
titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin_novo
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",0,23.0,1,0,21228,82.2667,S,NaN
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",0,47.0,1,0,W.E.P. 5734,61.1750,S,NaN
24,916,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",0,48.0,1,3,PC 17608,262.3750,C,NaN
26,918,1,"Ostby, Miss. Helene Ragnhild",0,22.0,0,1,113509,61.9792,C,NaN
28,920,1,"Brady, Mr. John Bertram",1,41.0,0,0,113054,30.5000,S,NaN


In [9]:
df_titanic_train.dropna(subset=['Age'],inplace=True)
df_titanic_train['Age'].isnull().sum()

df_titanic_train.dropna(subset=['Cabin'],inplace=True)
df_titanic_train['Cabin'].isnull().sum()

df_titanic_train.dropna(subset=['Embarked'],inplace=True)
df_titanic_train['Embarked'].isnull().sum()

0

In [11]:
df_titanic_train.isnull().values.any()
df_titanic_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

#### Ao analisar a tabela df_titanic_train, identificamos uma grande quantidade de valores nulos, o que poderia afetar a qualidade dos nossos modelos de machine learning. Para resolver esse problema, decidimos remover todos os valores nulos da tabela, garantindo assim que nossos dados estivessem limpos e prontos para serem utilizados nos modelos.

In [13]:
titanic_test.dropna(subset=['Age'],inplace=True)
titanic_test['Age'].isnull().sum()

titanic_test.dropna(subset=['Fare'],inplace=True)
titanic_test['Fare'].isnull().sum()

titanic_test.dropna(subset=['Cabin'],inplace=True)
titanic_test['Cabin'].isnull().sum()

0

In [15]:
titanic_test.isnull().values.any()
titanic_test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

#### Em seguida, realizei o processo de limpeza de dados na tabela de teste (titanic_test), onde excluí todos os valores nulos para garantir a consistência e a qualidade dos dados. Essa etapa foi fundamental para preparar os dados para a avaliação do modelo e garantir que as previsões fossem precisas e confiáveis.

In [17]:
df_titanic_train.info()
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 183 entries, 1 to 889
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  183 non-null    int64  
 1   Survived     183 non-null    int64  
 2   Pclass       183 non-null    int64  
 3   Name         183 non-null    object 
 4   Sex          183 non-null    object 
 5   Age          183 non-null    float64
 6   SibSp        183 non-null    int64  
 7   Parch        183 non-null    int64  
 8   Ticket       183 non-null    object 
 9   Fare         183 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     183 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 18.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 12 to 414
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  87 non-null     int64  
 1   Pclass       87 no

In [19]:
label_encoder = LabelEncoder()
df_titanic_train['Sex'] = label_encoder.fit_transform(df_titanic_train['Sex'])
titanic_test['Sex'] = label_encoder.fit_transform(titanic_test['Sex'])

#### Utilizamos o LabelEncoder para realizar a transformação da variável categórica 'Sex' em uma variável numérica. Após a aplicação do LabelEncoder, os valores foram mapeados da seguinte forma: 'female' foi atribuído o valor 0 e 'male' foi atribuído o valor 1. Essa transformação é fundamental para que os algoritmos de machine learning possam processar corretamente as variáveis categóricas.

In [21]:
df_titanic_train['Cabin_novo'] = df_titanic_train['Cabin'].notna().astype(int)
titanic_test['Cabin_novo'] = df_titanic_train['Cabin'].notna().astype(int)

df_titanic_train = df_titanic_train.drop(columns=['Cabin'])
titanic_test = titanic_test.drop(columns=['Cabin'])

#### No conjunto de treinamento (df_titanic_train), criamos uma nova coluna chamada 'Cabin_novo' que realiza uma verificação importante: ela identifica se o valor na coluna 'Cabin' é nulo ou não. Se o valor não for nulo, ou seja, se houver alguma informação registrada na cabine, a coluna 'Cabin_novo' recebe o valor 1; por outro lado, se o valor for nulo, indicando a ausência de informação sobre a cabine, a coluna 'Cabin_novo' recebe o valor 0.

In [223]:
df = ['Sex','Cabin_novo', 'Fare', 'Pclass']

In [225]:
X = df_titanic_train[df]
y = df_titanic_train['Survived']

X_test = titanic_test[df]

In [27]:
model_xgboost = xgb.XGBClassifier()
model_xgboost.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [29]:
model2_y_pred = model_xgboost.predict(X)

In [31]:
xgboost = classification_report(y, model2_y_pred)
print("Ralatorio de Classificação:")
print(xgboost)

Ralatorio de Classificação:
              precision    recall  f1-score   support

           0       0.88      0.85      0.86        60
           1       0.93      0.94      0.94       123

    accuracy                           0.91       183
   macro avg       0.90      0.90      0.90       183
weighted avg       0.91      0.91      0.91       183



#### Nosso projeto alcançou uma acurácia impressionante de 91% nos dados previstos pelo modelo XGBoost, o que é um resultado extremamente positivo e promissor. Essa alta taxa de acerto demonstra que o modelo foi capaz de aprender e generalizar bem os padrões presentes nos dados, tornando-o uma ferramenta valiosa para fazer previsões precisas. Com essa performance, acreditamos que nosso modelo tem grande potencial para ser bem-sucedido na plataforma Kaggle, onde podemos compartilhá-lo com a comunidade e compará-lo com outros modelos. Estamos confiantes de que essa acurácia de 91% será um diferencial importante para nossa submissão no Kaggle.

## Xgboost 2

In [33]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X, y)

In [35]:
scaler = StandardScaler()
X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test)

In [37]:
print("\nDistribuição das classes balanceadas:")
print(y_train_balanced.value_counts())


Distribuição das classes balanceadas:
Survived
1    123
0    123
Name: count, dtype: int64


#### Após realizar o balanceamento da nossa base de dados utilizando a técnica SMOTE, vamos aplicar o modelo XGBoost para avaliar seu desempenho e comparar os resultados com o nosso primeiro modelo. Isso nos permitirá verificar se o balanceamento das classes teve um impacto positivo na performance do modelo e identificar possíveis melhorias. Com essa comparação, poderemos determinar se o uso do SMOTE foi eficaz em melhorar a capacidade do modelo de fazer previsões precisas e tomar decisões informadas com base nos dados

In [167]:
grid_Xgboost = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 200],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'learning_rate': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

max_depth: É a profundidade máxima da árvore de decisão. Quanto maior a profundidade, mais complexa a árvore e mais capaz de aprender padrões nos dados.
- Valores: [4, 6, 8]
- Interpretação: O modelo irá testar árvores com profundidades de 4, 6 e 8 níveis.
n_estimators: É o número de árvores de decisão que serão combinadas para formar o modelo XGBoost. Quanto mais árvores, mais robusto o modelo, mas também mais lento para treinar.
- Valores: [10, 20, 30]
- Interpretação: O modelo irá testar conjuntos de 10, 20 e 30 árvores de decisão.
min_samples_split: É o número mínimo de amostras necessárias para dividir um nó da árvore de decisão. Isso ajuda a evitar overfitting.
- Valores: [2, 5, 10]
- Interpretação: O modelo irá testar valores de 2, 5 e 10 amostras mínimas para dividir um nó.
min_samples_leaf: É o número mínimo de amostras necessárias para estar em um nó folha da árvore de decisão. Isso também ajuda a evitar overfitting.
- Valores: [1, 2, 4]
- Interpretação: O modelo irá testar valores de 1, 2 e 4 amostras mínimas para estar em um nó folha.


In [201]:
XGB2 = xgb.XGBClassifier()

In [203]:
XGB2.fit(X_train_balanced_scaled, y_train_balanced)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [193]:
grid_xgboost = GridSearchCV(
    estimator=XGB,
    param_grid=grid_Xgboost,
    scoring='accuracy',
    cv=5,
    n_jobs=-1
)

In [173]:
grid_xgboost.fit(X_train_balanced_scaled, y_train_balanced)

C:\Users\User\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:08:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "max_features", "min_samples_leaf", "min_samples_split" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=Non...
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.8, 1.0], 'max_depth': [3, 5, 7],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [175]:
best_params = grid_xgboost.best_params_
print(f"Melhores Hiperparâmetros: {best_params}")

Melhores Hiperparâmetros: {'colsample_bytree': 0.8, 'learning_rate': 1.0, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


- colsample_bytree: 0.8
- Isso significa que o modelo irá amostrar 80% das colunas (features) para cada árvore de decisão. Isso ajuda a reduzir a dimensionalidade do problema e evitar overfitting.
- learning_rate: 1.0
- Isso é um valor alto para a taxa de aprendizado, o que significa que o modelo irá aprender rapidamente com os dados. No entanto, isso também pode levar a overfitting se o modelo for muito complexo.
- max_depth: 3
- Isso significa que o modelo irá construir árvores de decisão com uma profundidade máxima de 3 níveis. Isso é um valor razoável, pois permite que o modelo capture padrões nos dados sem se tornar muito complexo.
- max_features: 'sqrt'
- Esse parâmetro não é válido para o XGBoost. É provável que você tenha incluído esse parâmetro por engano. No XGBoost, o parâmetro equivalente é colsample_bytree, que você já definiu como 0.8.
- min_samples_leaf: 1
- Esse parâmetro não é válido para o XGBoost. É provável que você tenha incluído esse parâmetro por engano. No XGBoost, o parâmetro equivalente é min_child_weight, que controla o peso mínimo necessário para criar um novo nó na árvore de decisão.
- min_samples_split: 2
- Esse parâmetro não é válido para o XGBoost. É provável que você tenha incluído esse parâmetro por engano. No XGBoost, não há um parâmetro equivalente direto.
- n_estimators: 50
- Isso significa que o modelo irá combinar 50 árvores de decisão para fazer previsões. Isso é um valor razoável, pois permite que o modelo capture padrões nos dados sem se tornar muito lento.

In [177]:
Model_xgboost = grid_xgboost.best_estimator_
y_pred_xgboost = Model_xgboost.predict(X_train_balanced_scaled)

In [183]:
Relatorio = classification_report(y_train_balanced, y_pred_xgboost)
print("Relatorio de Classificação:")
print(Relatorio)

Relatorio de Classificação:
              precision    recall  f1-score   support

           0       0.91      0.94      0.93       123
           1       0.94      0.91      0.93       123

    accuracy                           0.93       246
   macro avg       0.93      0.93      0.93       246
weighted avg       0.93      0.93      0.93       246



In [207]:
y_xgboost = XGB2.predict(X_test_scaled)

In [209]:
y_xgboost

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1])

In [213]:
SUB_xgboost = pd.Series(y_xgboost, index=titanic_test['PassengerId'], name = 'Survived')

In [215]:
SUB_xgboost

PassengerId
904     1
906     1
916     1
918     1
920     1
       ..
1296    0
1297    1
1299    0
1303    1
1306    1
Name: Survived, Length: 87, dtype: int32

In [238]:
SUB_xgboost.to_csv("xgboost.csv", header=True)